In [602]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn
%pip install zemberek-python
%pip install trnltk
%pip install git+https://github.com/emres/turkish-deasciifier.git
%pip install transformers datasets
%pip install torch
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement poyraz (from versions: none)
ERROR: No matching distribution found for poyraz


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/emres/turkish-deasciifier.git to c:\users\cemke\appdata\local\temp\pip-req-build-2fi4_6eh
  Resolved https://github.com/emres/turkish-deasciifier.git to commit 665154c734b09485c3d11ce0038cd121dd109594
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may 

  Running command git clone --filter=blob:none --quiet https://github.com/emres/turkish-deasciifier.git 'C:\Users\cemke\AppData\Local\Temp\pip-req-build-2fi4_6eh'


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [619]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import logging
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
from trnlp import TrnlpWord
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, TextClassificationPipeline
from transformers import pipeline
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [627]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]   
            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]            
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
                
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="mixed", lang='tr')
            results=[]
            for item in cursor.items(10):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            DataSet.to_csv('tw.csv')
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            Dataset=pd.read_csv('tw.csv')                   
            getTweets.TweetClear(DataSet)
            getTweets.SentenceNormalization(DataSet)
            getTweets.TweetTranslate(DataSet)
            getTweets.KelimeAyristirma(DataSet) 
            getTweets.ToxicSentiment(DataSet)
            getTweets.TextblobSentiment(DataSet)

    def TextblobSentiment(DataSet):
        try:
            print ("Duygu Analizi Yapılıyor...")
            Veri=DataSet
            Veri["Tür"] = Veri["Tweet_Ayrı"].apply(getTweets.getPolarity)
            Veri["Yoğunluk"] = Veri["Tweet_Ayrı"].apply(getTweets.getSubjectivity)
            Veri.to_csv('textblob_eng_duygu.csv',index=False)
            print("Duygu Analizi Yapıldı!")                        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    """def TrSentiment(DataSet):
        try:
            print("Türkçe Duygu Analizi Yapılıyor...")
            for example,index in DataSet["Tweet_Degistirilmis"]:
                DataSet["TR_A"][index] = simple_token(example, sw=stopwords)
            print("Türkçe Duygu Analizi Yapıldı!")
        except Exception as err:
            print('Error: {}'.format(err)) """
    def TweetClear(DataSet):
        try:
            data=DataSet
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+|#[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|$[a-zA-ZğüşöçıİĞÜŞÖÇ]+|@[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ_]+|[,.^_$*%-;鶯!?’"“”:=|…#]+|\n+|RT[\s]+|amp[\s]+|(^gt)')
            for index in range(len(data["Tweet_Degistirilmis"])):        
                data["Tweet_Degistirilmis"][index]=pattern.sub(' ', data["Tweet_Degistirilmis"][index])    
                data["Tweet_Degistirilmis"][index]=data["Tweet_Degistirilmis"][index].lower()              
            data["Tweet_Degistirilmis"]=data["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return data
        except Exception as err:
            print('Error: {}'.format(err))  
                              
    def TweetTranslate(DataSet):
        try:
            data=DataSet
            print ("Tweetler Çeviriliyor...")
            for index in range(len(data["Tweet_Degistirilmis"])):
                Ceviri_Blob=TextBlob(data["Tweet_Degistirilmis"][index])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                data["Tweet_Ceviri"][index]=Eng_Blob.lower()                
            print ("Tweetler Çevirildi!")                      
            return data        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    def SentenceNormalization(DataSet):
        try:
            data=DataSet
            print("Cümle Normalizasyonu Yapılıyor...")
            logger = logging.getLogger(__name__)            
            morphology = TurkishMorphology.create_with_defaults()
            normalizer = TurkishSentenceNormalizer(morphology)
            for index in range(len(data["Tweet_Degistirilmis"])):                
                data["Tweet_Degistirilmis"][index]= normalizer.normalize(data["Tweet_Degistirilmis"][index])
            print("Cümle Normalizasyonu Yapıldı!")
            sc = TurkishSpellChecker(morphology)
            return data
        except Exception as err:
            print('Error: {}'.format(err))
            
    def KelimeAyristirma(DataSet):
        try:
            data=DataSet
            stop = stopwords.words('english')
            data["Tweet_Ayrı"] = data["Tweet_Ceviri"].str.split()
            data["Tweet_Ayrı"] = data["Tweet_Ayrı"].apply(lambda x: [item for item in x if item not in stop])
            data["Tweet_Ayrı"] = data["Tweet_Ayrı"].str.join(" ")
            return data
        except Exception as err:
            print('Error: {}'.format(err))
    
    def ToxicSentiment(DataSet):
        try:
            data=DataSet
            print("Toxic Cümle Analizi Yapılıyor...")
            model_path ="martin-ha/toxic-comment-model"
            tokenizer=AutoTokenizer.from_pretrained(model_path)
            model=AutoModelForSequenceClassification.from_pretrained(model_path)
            pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
            for index in range(len(data["Tweet_Ceviri"])):
                sonuc=pipeline(str(data["Tweet_Toxic"][index]))
                if sonuc[0]['label']=="toxic":
                    data["Tweet_Toxic"][index]="Toxic"
                else:
                    data["Tweet_Toxic"][index]="Toxic değil"    
            print("Toxic Cümle Analizi Yapıldı!")
            return data  
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "%75 - %100"
            elif sub >=0.5 and sub <0.75:
                return "%50 - %75"
            elif sub >=0.25 and sub <0.50:
                return "%25 - %50"
            else:
                return "0 - 25%"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.3:
                return "Pozitif"
            elif pol<-0.3:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

In [628]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Cümle Normalizasyonu Yapılıyor...
2023-01-12 05:30:10,185 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 9.684077739715576

Cümle Normalizasyonu Yapıldı!
Tweetler Çeviriliyor...
Tweetler Çevirildi!
Toxic Cümle Analizi Yapılıyor...
Error: 'Tweet_Toxic'
Duygu Analizi Yapılıyor...
Duygu Analizi Yapıldı!


In [629]:
Veri=pd.read_csv('textblob_eng_duygu.csv')
Veri.head()

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tür,Yoğunluk
0,T.C. Tarım ve Orman Bakanlığı,2023-01-02 13:23:39+00:00,TarımCebimde mobil uygulaması ile tarımla ilgili birçok bilgi ve işlem artık bir tık uzağınızda👇🏻\n\nApp Store… https://t.co/J3lWZUoPge,tarımcebimde mobil uygulaması ile tarımla ilgili birçok bilgi ve işlem artık bir tık uzağınızda appstore,many information and transactions related to agriculture with the mobile application in agriculture now are now a click away appstore,many information transactions related agriculture mobile application agriculture click away appstore,Nötr,%25 - %50
1,Türk Polis Teşkilatı,2023-01-03 08:39:07+00:00,#UYUMA uygulaması ile karşılaştığınız uyuşturucu ticaretini doğrudan ve en hızlı şekilde bildirin!\n\nUygulamayı indi… https://t.co/8WaLKZsD5j,uygulaması ile karşılaştığınız uyuşturucu ticaretini doğrudan ve en hızlı şekilde bildirin uygulamayı indi,report the drug trade directly and fastest in the fastest way.,report drug trade directly fastest fastest way.,Nötr,%25 - %50
2,müthiş indirim,2023-01-11 16:35:28+00:00,Akıllı Saatler Kısa Süreliğine İndirimde Kaçırmayınn⌚️🔥\n\n🔗 https://t.co/tfJO1ZAeQb https://t.co/yx5JcK4Svl,akıllı saatler kısa süreliğine indirimde kaçırmayın,smart watches do not miss for a short time,smart watches miss short time,Nötr,%25 - %50
3,Cigdem Kaya,2023-01-12 02:27:15+00:00,RT @CanSelcuki: Ocak ayı anketi tamamlandı. \n\n- AK Parti yükseliş trendi devam ediyor\n- Kararsızlarda anlamlı bir düşüş var\n- EYT toplumun…,ocak ayı anketi tamamlandı ak parti yükseliş trendi devam ediyor kararsızlarda anlamlı bir düşüş var eyt toplumun,"january survey was completed, the ak party rise trend continues, there is a significant decrease in the undecided.","january survey completed, ak party rise trend continues, significant decrease undecided.",Pozitif,%75 - %100
4,Halit Çelik,2023-01-12 02:26:55+00:00,Benim de oynadığım Futbol Patronu iPhone oyununu bir dene https://t.co/41pSGJjSfc,benim de oynadığım futbol patronu ıphone oyununu bir dene,try the football boss i play a game of iphone.,try football boss play game iphone.,Negatif,%25 - %50


In [630]:
Veri.tail()

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tür,Yoğunluk
5,Oliver Jack,2023-01-12 02:26:09+00:00,https://t.co/s8mWtQwR9G #Buca yalnız zekiyim canlı ise Reşat hırsızların,yalnız zekîyim canlı ise reşat hırsızların,"if i am alone, if you are alive, reşat thieves","alone, alive, reşat thieves",Nötr,%25 - %50
6,Hz.Mert,2023-01-12 02:24:43+00:00,@Apple 14 ü çok başarılı yapamamışsınız siz iyisi mi 15 i çıkarın da onu alalım..,u çok başarılı yapamamışsınız siz iyisi mi ı çıkarın da onu alalım,you haven't done it very successful?,done successful?,Pozitif,%75 - %100
7,Oliver Jack,2023-01-12 02:18:46+00:00,https://t.co/25IYCAwUzf #Buca bilene yakarsın gidiyor bilirim fotograf Kitap bile degil,bilene yakarsın gidiyor bilirim fotoğraf kitap bile değil,"you know, i know, i know the photo is not even a book","know, know, know photo even book",Nötr,0 - 25%
8,Oliver Jack,2023-01-12 02:11:21+00:00,https://t.co/8y7hKi9J4S Gibi sigara #Buca hastaneler,gibi sigara hastaneler,cigarette hospitals like,cigarette hospitals like,Nötr,0 - 25%
9,Oliver Jack,2023-01-12 02:03:57+00:00,https://t.co/PKozS01mG2 Dinleyenler Düşünmek daha beklemek resmen göre Schopenhauer tohumudur #Buca bilir iyi,dinleyenler düşünmek daha beklemek resmen göre schopenhauer tohumudur bilir iyi,listening to think more to wait more than the schopenhauer seed knows good.,listening think wait schopenhauer seed knows good.,Pozitif,%50 - %75


In [631]:
"""#Argo cümle
model_path ="martin-ha/toxic-comment-model"
tokenizer=AutoTokenizer.from_pretrained(model_path)
model=AutoModelForSequenceClassification.from_pretrained(model_path)
pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
text="bu cümle toksik bir cümle değil amınakoyayım"
b_text=TextBlob(text)
e_text=b_text.translate(from_lang='tr', to='en')  
sonuc=pipeline(str(e_text))
if sonuc[0]['label']=="toxic":
    print ("Toxic")
else:
    print ("Toxic değil")"""

'#Argo cümle\nmodel_path ="martin-ha/toxic-comment-model"\ntokenizer=AutoTokenizer.from_pretrained(model_path)\nmodel=AutoModelForSequenceClassification.from_pretrained(model_path)\npipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)\ntext="bu cümle toksik bir cümle değil amınakoyayım"\nb_text=TextBlob(text)\ne_text=b_text.translate(from_lang=\'tr\', to=\'en\')  \nsonuc=pipeline(str(e_text))\nif sonuc[0][\'label\']=="toxic":\n    print ("Toxic")\nelse:\n    print ("Toxic değil")'

In [632]:
help(tweepy.Cursor)

Help on class Cursor in module tweepy.cursor:

class Cursor(builtins.object)
 |  Cursor(method, *args, **kwargs)
 |  
 |  :class:`Cursor` can be used to paginate for any :class:`API` methods that
 |  support pagination
 |  
 |  Parameters
 |  ----------
 |  method
 |      :class:`API` method to paginate for
 |  args
 |      Positional arguments to pass to ``method``
 |  kwargs
 |      Keyword arguments to pass to ``method``
 |  
 |  Methods defined here:
 |  
 |  __init__(self, method, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  items(self, limit=inf)
 |      Retrieve the items in each page/request
 |      
 |      Parameters
 |      ----------
 |      limit
 |          Maximum number of items to iterate over
 |      
 |      Returns
 |      -------
 |      ItemIterator
 |          Iterator to iterate through items
 |  
 |  pages(self, limit=inf)
 |      Retrieve the page for each request
 |      
 |      Parameters
 |      ---------

In [633]:
#Eksik Cümle
unmasker=pipeline('fill-mask', model='xlm-roberta-base')
